In [ ]:
!pip install openpyxl requests
!pip install evaluate
!pip install sacrebleu
!pip install codebleu[all]
!pip install tree-sitter-python==0.21

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.2/544.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import json
import requests
import os
import time
import pandas as pd
import evaluate
import json
import re
import openpyxl
from openpyxl.utils import get_column_letter
from codebleu import calc_codebleu
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [ ]:
os.environ["GROQ_API_KEY"] = ""

# **Scenario 1 (Requirement Pendek)**

In [ ]:
# --- Konfigurasi ---
EXCEL_FILE = "scenario1.xlsx"
SHEET_NAME = "Lembar1"
REQUIREMENT_COLUMN = "Requirements"
RESOURCE_COLUMN = "Resources"
NAMETEST_COLUMN = "Name Test"
OUTPUT_COLUMN = "Generated Test"

GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"
GROQ_MODEL = "llama3-70b-8192"
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

# --- Fungsi untuk Memanggil Groq API  ---
def call_groq_api(prompt, retries=3, delay=5):
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": GROQ_MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 500,
        "temperature": 0.7
    }

    for attempt in range(retries):
        try:
            response = requests.post(GROQ_API_URL, headers=headers, json=data)
            if response.status_code == 429:
                print("Rate limit hit. Waiting before retry...")
                time.sleep(delay)
                continue
            response.raise_for_status()
            return response.json()["choices"][0]["message"]["content"]
        except requests.exceptions.RequestException as e:
            print(f"Error calling Groq API (attempt {attempt+1}): {e}")
            time.sleep(delay)
    return None

# --- Fungsi Utama ---
def generate_tests_from_excel(excel_file, sheet_name, requirement_column, output_column, resource_column, nametest_column):
    try:
        workbook = openpyxl.load_workbook(excel_file)
        sheet = workbook[sheet_name]
    except FileNotFoundError:
        print(f"Error: Excel file '{excel_file}' not found.")
        return
    except KeyError:
        print(f"Error: Sheet '{sheet_name}' not found in Excel file.")
        return

    header_row = 1
    requirement_col_index = None
    output_col_index = None
    resource_col_index = None
    nametest_col_index = None

    for col_num, cell in enumerate(sheet[header_row], start=1):
        if cell.value == requirement_column:
            requirement_col_index = col_num
        if cell.value == output_column:
            output_col_index = col_num
        if cell.value == resource_column:
            resource_col_index = col_num
        if cell.value == nametest_column:
            nametest_col_index = col_num

    if requirement_col_index is None or resource_col_index is None:
        print(f"Error: Required columns not found.")
        return

    for row_num in range(header_row + 1, sheet.max_row + 1):
        resource = sheet.cell(row=row_num, column=resource_col_index).value
        requirement = sheet.cell(row=row_num, column=requirement_col_index).value
        nametest = sheet.cell(row=row_num, column=nametest_col_index).value


        if requirement and resource and nametest:
            prompt = f"""You are a professional software tester. Based on the following API requirement, write a complete and structured unit test using Pytest.

Follow these rules:
1. Use clear and descriptive test function names.
2. Use the `token_client` fixture to send authenticated requests.
3. Assert the response status code and key data in the response.
4. Cover both success and failure scenarios if applicable.
5. Use the existing object fixture if provided (e.g., `organizer`, `user`, `bundle`, etc.).
6. Generate ONLY ONE `pytest` function that tests all possible cases in this requirement.
7. ABSOLUTELY DO NOT include any explanation, comments, or text outside the code.
8. Output MUST be ONLY the raw Python code enclosed in triple backticks.
9. The test function name MUST follow this template: {nametest}

Resource: {resource}

Requirement:
{requirement}

Example format:

```python
def test_<name>(...):
    ...
```"""
            print(f"\nProcessing requirement at row {row_num} for resource '{resource}'...")
            time.sleep(2.5)
            generated_test = call_groq_api(prompt)
            if generated_test:
                print(f"✅ Generated test for: {requirement}")
                if output_col_index:
                    code_block = generated_test.strip()
                    if "```python" in code_block:
                        code_block = code_block.split("```python")[1].split("```")[0].strip()
                    elif "```" in code_block:
                        code_block = code_block.split("```")[1].split("```")[0].strip()

                    sheet.cell(row=row_num, column=output_col_index).value = code_block

    if output_col_index:
        workbook.save(excel_file)
        print(f"\n✅ All generated tests saved to '{output_column}' column in '{excel_file}'.")

# --- Jalankan ---
if __name__ == "__main__":
    generate_tests_from_excel(EXCEL_FILE, SHEET_NAME, REQUIREMENT_COLUMN, OUTPUT_COLUMN, RESOURCE_COLUMN, NAMETEST_COLUMN)



Processing requirement at row 2 for resource 'Organizers'...
✅ Generated test for: Returns a list of all organizers the authenticated user/token has access to.

Processing requirement at row 3 for resource 'Organizers'...
✅ Generated test for: Returns information on one organizer account, identified by its slug.

Processing requirement at row 4 for resource 'Organizers'...
✅ Generated test for: Get current values of organizer settings. Permission required: “Can change organizer settings”

Processing requirement at row 5 for resource 'Organizers'...
✅ Generated test for: Updates organizer settings. Note that PUT is not allowed here, only PATCH.

Processing requirement at row 6 for resource 'Events'...
✅ Generated test for: Returns a list of all events within a given organizer the authenticated user/token has access to. Permission required: “Can change event settings”

Processing requirement at row 7 for resource 'Events'...
✅ Generated test for: Returns information on one event, identi

# **Scenario 2 (Requirement Panjang)**

In [ ]:
# --- Konfigurasi ---
JSON_FILE = "scenario2.json"
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"
GROQ_MODEL = "llama3-70b-8192"
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
EXCEL_PATH = "/content/scenario2.xlsx"

# --- Fungsi untuk Memanggil Groq API ---
def call_groq_api(prompt, retries=3, delay=5):
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": GROQ_MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 500,
        "temperature": 0.7
    }

    for attempt in range(retries):
        try:
            response = requests.post(GROQ_API_URL, headers=headers, json=data)
            if response.status_code == 429:
                print("Rate limit hit. Waiting before retry...")
                time.sleep(delay)
                continue
            response.raise_for_status()
            return response.json()["choices"][0]["message"]["content"]
        except requests.exceptions.RequestException as e:
            print(f"Error calling Groq API (attempt {attempt+1}): {e}")
            time.sleep(delay)
    return None

# --- Fungsi Utama ---
def generate_tests_from_json_to_excel(json_file, excel_path):
    try:
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"❌ File '{json_file}' tidak ditemukan.")
        return

    wb = openpyxl.load_workbook(excel_path)
    ws = wb.active

    # Cari kolom "Generated Test"
    header_row = ws[1]
    headers = {cell.value.strip(): idx + 1 for idx, cell in enumerate(header_row)}
    if "Generated Test" not in headers:
        print("❌ Kolom 'Generated Test' tidak ditemukan.")
        return
    generated_test_col = headers["Generated Test"]

    row_index = 2  # mulai dari baris ke-2 (asumsikan baris pertama adalah header)
    for section_name, section_data in data.items():
        description = section_data.get("description", "")
        endpoints = section_data.get("endpoints", {})

        for endpoint_path, endpoint_details in endpoints.items():
            method = endpoint_path.split()[0]
            url = endpoint_path.split()[1]
            name_test = endpoint_details.get("name_test", "def test_example(...):")

            requirement = f"""You are a professional software tester. Based on the following API requirement, write a complete and structured unit test using Pytest.

Follow these rules:
1. Use clear and descriptive test function names.
2. Use the `token_client` fixture to send authenticated requests.
3. Assert the response status code and key data in the response.
4. Cover both success and failure scenarios if applicable.
5. Use the existing object fixture if provided (e.g., `organizer`, `user`, `bundle`, etc.).
6. Generate ONLY ONE `pytest` function that tests all possible cases in this requirement.
7. ABSOLUTELY DO NOT include any explanation, comments, or text outside the code.
8. Output MUST be ONLY the raw Python code enclosed in triple backticks.
9. The test function name MUST follow this template: {name_test}

Entity: {section_name}
Description: {description}

Endpoint: {endpoint_path}
Details: {json.dumps(endpoint_details, indent=2)}

Example format:

```python
def test_<name>(...):
    ...
```"""

            print(f"\n🔄 Processing: {endpoint_path}")
            time.sleep(2.5)
            result = call_groq_api(requirement)

            if result:
                code_block = result.strip()
                if "```python" in code_block:
                    code_block = code_block.split("```python")[1].split("```")[0].strip()
                elif "```" in code_block:
                    code_block = code_block.split("```")[1].split("```")[0].strip()

                ws.cell(row=row_index, column=generated_test_col).value = code_block
                print(f"✅ Generated test inserted at row {row_index}")

            row_index += 1

    wb.save(excel_path)
    print(f"\n✅ All generated tests have been saved in '{excel_path}'.")


# --- Jalankan ---
if __name__ == "__main__":
    generate_tests_from_json_to_excel(JSON_FILE, EXCEL_PATH)


🔄 Processing: GET /api/v1/organizers/
✅ Generated test inserted at row 2

🔄 Processing: GET /api/v1/organizers/(organizer)/
✅ Generated test inserted at row 3

🔄 Processing: GET /api/v1/organizers/(organizer)/settings/
✅ Generated test inserted at row 4

🔄 Processing: PATCH /api/v1/organizers/(organizer)/settings/
✅ Generated test inserted at row 5

🔄 Processing: GET /api/v1/organizers/(organizer)/events/
✅ Generated test inserted at row 6

🔄 Processing: GET /api/v1/organizers/(organizer)/events/(event)/
✅ Generated test inserted at row 7

🔄 Processing: POST /api/v1/organizers/(organizer)/events/
✅ Generated test inserted at row 8

🔄 Processing: POST /api/v1/organizers/(organizer)/events/(event)/clone/
✅ Generated test inserted at row 9

🔄 Processing: PATCH /api/v1/organizers/(organizer)/events/(event)/
✅ Generated test inserted at row 10

🔄 Processing: DELETE /api/v1/organizers/(organizer)/events/(event)/
✅ Generated test inserted at row 11

🔄 Processing: GET /api/v1/organizers/(orga

# **Evaluasi**

In [ ]:
chrf_evaluator = evaluate.load("chrf")
bleu_evaluator = evaluate.load("bleu")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def evaluate_code(filepath):
    codebleu_scores, bleu_scores = [], []
    chrf_scores = []

    print("===== START EVALUATION FROM EXCEL =====\n")

    # Load Excel file
    df = pd.read_excel(filepath)

    for i, row in df.iterrows():
        print(f"Data ke-{i}")

        reference_code = row.get("Reference Test")
        candidate_code = row.get("Generated Test")

        if pd.isna(reference_code) or pd.isna(candidate_code):
            print("⚠️ Skipping this example due to missing data.")
            continue

        try:
            if not str(reference_code).strip() or not str(candidate_code).strip():
                print("⚠️ Warning: reference_code or candidate_code is empty. Skipping evaluations.")
                codebleu_metric = {"codebleu": 0.0}
                bleu_metric = {"bleu": 0.0}
                chrf_metric = {"score": 0.0}
            else:
                codebleu_metric = calc_codebleu(
                    references=[[reference_code]],
                    predictions=[candidate_code],
                    lang="python",
                    weights=(0.25, 0.25, 0.25, 0.25),
                    tokenizer=None
                )
                bleu_metric = bleu_evaluator.compute(predictions=[candidate_code], references=[[reference_code]])
                chrf_metric = chrf_evaluator.compute(predictions=[candidate_code], references=[[reference_code]])

        except Exception as e:
            print(f"⚠️ Error in evaluation: {e}")
            continue

        bleu_score = bleu_metric.get('bleu', 0.0)
        codebleu_score = codebleu_metric.get('codebleu', 0.0)
        chrf_score = chrf_metric.get('score', 0.0)

        bleu_scores.append(bleu_score)
        chrf_scores.append(chrf_score)
        codebleu_scores.append(codebleu_score)

    def safe_average(scores):
        return sum(scores) / len(scores) if scores else 0.0

    return {
        "avg_codebleu": f"{safe_average(codebleu_scores)}",
        "avg_bleu": f"{safe_average(bleu_scores)}",
        "avg_chrf_scores": f"{safe_average(chrf_scores)}",
        "": "",
        "=============": "PERCENT ==================",
        "": "",
        "Avg CodeBLEU": f"{round(safe_average(codebleu_scores) * 100, 2)}%",
        "Avg BLEU": f"{round(safe_average(bleu_scores) * 100, 2)}%",
        "Avg chrF": f"{round(safe_average(chrf_scores), 2)}%",
    }

In [ ]:
filepath = "/content/scenario2.xlsx"

result_evaluate = evaluate_code(filepath)
print("\n ------------ RESULT EVALUATE ---------------")
print(json.dumps(result_evaluate, indent=4))

===== START EVALUATION FROM EXCEL =====

Data ke-0
Data ke-1
Data ke-2
Data ke-3
Data ke-4
Data ke-5
Data ke-6


Data ke-7
Data ke-8
Data ke-9
Data ke-10
Data ke-11
Data ke-12
Data ke-13
Data ke-14
Data ke-15
Data ke-16
Data ke-17
Data ke-18
Data ke-19
Data ke-20
Data ke-21
Data ke-22
Data ke-23
Data ke-24
Data ke-25
Data ke-26
Data ke-27
Data ke-28
Data ke-29
Data ke-30
Data ke-31
Data ke-32
Data ke-33
Data ke-34
Data ke-35
Data ke-36
Data ke-37
Data ke-38
Data ke-39
Data ke-40
Data ke-41
Data ke-42
Data ke-43
Data ke-44
Data ke-45
Data ke-46
Data ke-47
Data ke-48
Data ke-49
Data ke-50
Data ke-51
Data ke-52
Data ke-53
Data ke-54
Data ke-55
Data ke-56
Data ke-57
Data ke-58
Data ke-59
Data ke-60
Data ke-61
Data ke-62
Data ke-63
Data ke-64
Data ke-65
Data ke-66
Data ke-67
Data ke-68
Data ke-69
Data ke-70
Data ke-71
Data ke-72
Data ke-73
Data ke-74
Data ke-75
Data ke-76
Data ke-77
Data ke-78
Data ke-79
Data ke-80
Data ke-81
Data ke-82
Data ke-83
Data ke-84
Data ke-85
Data ke-86
Data ke-87
Data ke-88
Data ke-89
Data ke-90
Data ke-91
Data ke-92
Data ke-93
Data ke-94
Data ke-95
Data ke-96
Data ke-97
Da

Data ke-140
Data ke-141
Data ke-142
⚠️ Skipping this example due to missing data.
Data ke-143
Data ke-144
Data ke-145
Data ke-146


Data ke-147
Data ke-148
Data ke-149
Data ke-150
Data ke-151
Data ke-152
Data ke-153


Data ke-154
⚠️ Skipping this example due to missing data.
Data ke-155
Data ke-156
Data ke-157
Data ke-158
Data ke-159
Data ke-160
Data ke-161
Data ke-162


Data ke-163
Data ke-164
Data ke-165
Data ke-166
Data ke-167
Data ke-168
Data ke-169
Data ke-170
Data ke-171
Data ke-172
Data ke-173
Data ke-174
Data ke-175
Data ke-176
Data ke-177
Data ke-178
Data ke-179
Data ke-180
Data ke-181
Data ke-182
Data ke-183
Data ke-184
Data ke-185
Data ke-186
Data ke-187
Data ke-188
Data ke-189
Data ke-190
Data ke-191

 ------------ RESULT EVALUATE ---------------
{
    "avg_codebleu": "0.2870132856856345",
    "avg_bleu": "0.17299497324759983",
    "avg_chrf_scores": "42.9567825173456",
    "": "",
    "=============": "PERCENT ==================",
    "Avg CodeBLEU": "28.7%",
    "Avg BLEU": "17.3%",
    "Avg chrF": "42.96%"
}


In [ ]:
candidate_code = """
def test_organizer_list(token_client, organizer):
    response = token_client.get("/api/organizers")
    assert response.status_code == 200
    organizers = response.json()
    assert len(organizers) > 0
    for organizer_data in organizers:
        assert "id" in organizer_data
        assert "name" in organizer_data
        assert "description" in organizer_data
    # Test failure scenario: unauthorized access
    token_client2 = token_client._replace(headers={"Authorization": "Bearer invalid_token"})
    response2 = token_client2.get("/api/organizers")
    assert response2.status_code == 401
"""

reference_code = """
def test_organizer_list(token_client, organizer):
    resp = token_client.get('/api/v1/organizers/')
    assert resp.status_code == 200
    assert TEST_ORGANIZER_RES in resp.data['results']
"""

if not reference_code.strip() or not candidate_code.strip():
    print("Warning: reference_code or candidate_code is empty. Skipping evaluations.")
    codebleu = {"codebleu": 0.0}
    bleu_metric = {"bleu": 0.0}
    chrf_metric = {"score": 0.0}
else:
    codebleu = calc_codebleu(
        references=[[reference_code]],
        predictions=[candidate_code],
        lang="python",
        weights=(0.25, 0.25, 0.25, 0.25)
    )
    bleu_metric = bleu_evaluator.compute(predictions=[candidate_code], references=[[reference_code]])
    chrf_metric = chrf_evaluator.compute(predictions=[candidate_code], references=[[reference_code]])


def format_score(score, is_percentage_scale=False):
    if is_percentage_scale:
        return f"{score:.2f}%"
    return f"{score * 100:.2f}% ({score:.2f})"

print(f"--> BLEU\t: {format_score(bleu_metric.get('bleu', 0.0))}\n")
print(f"--> CodeBLEU\t: {format_score(codebleu.get('codebleu', 0.0))}\n")
print(f"--> chrF\t: {format_score(chrf_metric.get('score', 0.0), is_percentage_scale=True)}\n")


--> BLEU	: 19.50% (0.20)

--> CodeBLEU	: 45.06% (0.45)

--> chrF	: 47.68%

